## Preparation for FDW Crop Production Data Profiling
- [FEWS NET Data Warehouse (FDW)](https://fdw.fews.net/en/)
- [FDW API Guide](https://fdw.fews.net/en/docs/api_reference/api_reference.html)
- [FEWSNET Data Inventory](https://fdw.fews.net/dashboard/inventory/)

Donghoon Lee (donghoonlee@ucsb.edu)</br>
Revised at 2022.06.24

In [1]:
import os, sys, json
from itertools import product, compress, chain
from functools import reduce
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
import plotly
import plotly.graph_objects as go
import plotly.express as px

### 0. Temporarily read in local DF to test

In [6]:
df = pd.read_csv('/Volumes/Svalbard/LargeDatasets/FEWS_Data_Warehouse/Crop_production/FDW/allCrops_SSH_Africa.csv')

/Users/wanders7/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (6,7,9,10,33) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
# Retrieve all grain data using API
#host = 'https://fdw.fews.net'
#auth = tuple(json.loads(open('token.json', "r").read()))
#parameters = {
#    'format': 'json',
#    'product': 'R011',
#    'survey_type': 'crop:best'
#}
#endpoint = '/api/cropproductionindicatorvalue/'
#response = requests.get(host + endpoint, auth=auth, params=parameters, proxies={})
#response.raise_for_status()
#df = pd.DataFrame.from_records(response.json())

FileNotFoundError: [Errno 2] No such file or directory: 'token.json'

### 1. Central Product Classification (CPC) Version 2.1 (CPCV2) code - Grain products
- UN's CPC Version 2.1 document can be found at [here](https://digitallibrary.un.org/record/3900378?ln=en), [table](http://datalab.review.fao.org/datalab/caliper/web/classification-page/39), [pdf](https://unstats.un.org/unsd/classifications/unsdclassifications/cpcv21.pdf).
- code/broader category
    - 0: Agriculture, forestry and fishery products
    - 01: Products of agriculture, horticulture and market gardening
    - 011: Cereals
    - 0111: Wheat
    - 0112: Maize
    - 0113: Rice
    - 0114: Sorghum
    - 0115: Barley
    - 0116: Rye
    - 0117: Oats
    - 0118: Millet
    - 0119: Other cereals

In [7]:
# Create a grain_code table
cpcv_category_name = {
    '111': 'Wheat', 
    '112': 'Maize', 
    '113': 'Rice', 
    '114': 'Sorghum', 
    '115': 'Barley', 
    '116': 'Rye', 
    '117': 'Oats', 
    '118': 'Millet', 
}
grain_code = df[['cpcv2', 'cpcv2_description', 'product']].drop_duplicates().sort_values('cpcv2').reset_index(drop=True)
grain_code['cpcv2_category'] = grain_code['cpcv2'].apply(lambda x: x[2:5])
grain_code['product_category'] = grain_code['cpcv2_category'].replace(cpcv_category_name)
grain_code.loc[grain_code['cpcv2'] == 'R01190AA', 'product_category'] = 'Teff'
grain_code.loc[grain_code['cpcv2'] == 'R01190AB', 'product_category'] = 'Fonio'
grain_code.loc[grain_code['cpcv2'] == 'R01190XX', 'product_category'] = 'Mixed'
fn_out = './data/crop/grain_cpcv2_code.hdf'
grain_code.to_hdf(fn_out, 'df')
print('%s is saved.' % fn_out)
grain_code

./data/crop/grain_cpcv2_code.hdf is saved.


,cpcv2,cpcv2_description,product,cpcv2_category,product_category
0,R01111AA,Wheat planting seed,Wheat planting seed,111,Wheat
1,R01112AA,"Wheat grain, other",Wheat Grain,111,Wheat
2,R01121AA,Maize/corn planting seed,Maize/corn planting seed,112,Maize
3,R01122AA,"Maize grain (corn), other",Maize (Corn),112,Maize
4,R01122AB,"Maize grain (corn), other, yellow",Maize Grain (Yellow),112,Maize
...,...,...,...,...,...
69,R01709AM,Macuna pruriens,Velvet bean,170,170
70,R01709AN,Cajanus cajan,Pigeon pea (hybrid),170,170
71,R01709AO,Lablab purpureus,Bean (Hyacinth),170,170
72,R01709AT,Geocarpa groundnut,Geocarpa groundnut,170,170


### 2. African countries where FDW grain data is available

In [8]:
sub = df[df['status'] == 'Collected']
sub.pivot_table(index='country',columns='indicator',values='value',aggfunc=len,fill_value=0)

indicator,Area Harvested,Area Planted,Quantity Produced,Yield
country,,,,
Angola,1587,2235,2230,2224
Benin,0,7252,7229,7220
Burkina Faso,0,7362,7339,7299
Burundi,0,1804,6559,1802
Cameroon,0,4328,4511,4298
Cape Verde,0,58,65,58
Central African Republic,132,0,132,132
Chad,2459,0,2440,2620
"Congo, The Democratic Republic of the",0,244,1632,244


In [9]:
# Comparison with a world shapefile
country_fdw = df['country'].unique()
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
lims = world[world.continent == 'Africa'].total_bounds[[0,2,1,3]]
world['name'] = world['name'].replace({
    'S. Sudan': 'South Sudan',
    'Tanzania': 'Tanzania, United Republic of',
    'Central African Rep.': 'Central African Republic',
    'Somaliland': 'Somalia'
})
world = world.dissolve(by='name',as_index=False)

# Select African countries
country = world.loc[world['name'].isin(country_fdw)]
country_africa = country[country['continent'] == 'Africa']
num_country_africa = country_africa.shape[0]
print('FDW grain data exists in %d African countries.' % num_country_africa)
world['fdw_data'] = np.nan
world.loc[world.name.isin(country_africa['name']), 'fdw_data'] = 1
geojson = json.loads(world[['name','geometry']].to_json())
assert world.name.isin(country_fdw).sum() == len(country_fdw)

# Mapping
fig = go.Figure(data=go.Choropleth(
    locations = world.name,
    z = world.fdw_data,
    geojson=geojson,
    featureidkey='properties.name',
    marker_line_width=1,
    marker_line_color='black',zmin=0, zmax=1,
    colorbar = None,
))
fig.update_traces(showscale=False)
fig.update_geos(visible=False, resolution=50,
                showcountries=True, countrycolor="grey",
                lonaxis_range=lims[:2],
                lataxis_range=lims[2:4],
                showframe=False,
               )
fig.update_layout(
    width=600, height=600,
    margin={"r":0,"t":0,"l":0,"b":20},
    font_size=14,
    dragmode=False
)
fig.add_annotation(
    xref='paper',yref='paper',
    x=0, y= -0.03,
    text='*%d African countries where FDW grain data is available' % num_country_africa,
    align="left",
    showarrow=False,
    font = {'family':'arial','size':15, 'color':'dimgrey'},
)
# fig.show()
fn_save = './figures/map_fdw_available.png'
fig.write_image(fn_save)
print('%s is saved.' % fn_save)
fn_out = './data/shapefile/country_fdw_grain_data_available.shp'
world.to_file(fn_out)
print('%s is saved.' % fn_out)

FDW grain data exists in 33 African countries.


AssertionError: 

![image](https://github.com/chc-ucsb/gscd/blob/main/figures/map_fdw_available.png?raw=true)

In [10]:
world.loc[world['fdw_data'].notna(),'name'].reset_index(drop=True)

0                           Angola
1                            Benin
2                     Burkina Faso
3                          Burundi
4                         Cameroon
5         Central African Republic
6                             Chad
7                    Côte d'Ivoire
8                         Ethiopia
9                           Gambia
10                           Ghana
11                          Guinea
12                           Kenya
13                         Lesotho
14                         Liberia
15                      Madagascar
16                          Malawi
17                            Mali
18                      Mauritania
19                      Mozambique
20                           Niger
21                         Nigeria
22                          Rwanda
23                         Senegal
24                    Sierra Leone
25                         Somalia
26                     South Sudan
27                           Sudan
28    Tanzania, Unit

### 3. Download all FEWS NET shapefiles

In [2]:
# ISO codes of all countries from the African shapefile
df1 = gpd.read_file('./data/shapefile/fewsnet/FEWSNET_Admin1.shp')
df2 = gpd.read_file('./data/shapefile/fewsnet/FEWSNET_Admin2.shp')
country_code1 = df1.loc[df1['COUNTRY'].notna(),['COUNTRY','ADMIN0']].drop_duplicates().reset_index(drop=True)
country_code2 = df2.loc[df2['COUNTRY'].notna(),['COUNTRY','ADMIN0']].drop_duplicates().reset_index(drop=True)
fnid_code = pd.concat([df1['FNID'].apply(lambda x: str(x)[:8]), df2['FNID'].apply(lambda x: str(x)[:8])],axis=0)
fnid_code = fnid_code.unique()

# # Extract "ZA_Admin1_1994.shp" from "FEWSNET_Admin1.shp"
# sub = df1[df1['FNID'].apply(lambda x: str(x)[:8] == 'ZA1994A1')].reset_index(drop=True)
# fn_out = './data/shapefile/fewsnet/ZA_Admin1_1994.shp'
# sub.to_file(fn_out)
# print('%s is saved.' % fn_out)

# # Download shapefiles of administrative boundaries from FEWS NET
# path_url = 'https://fews.net/data_portal_download/download?data_file_path=http%3A//shapefiles.fews.net.s3.amazonaws.com/ADMIN/'
# path_dir = './data/shapefile/fewsnet/'
# comb = product(list(country_code1['COUNTRY'].unique()),[1,2],list(np.arange(1950,2023)))
# for (code, level, year) in comb:
#     shape_name = '%s_Admin%d_%d.zip' % (code, level, year)
#     fn_url = os.path.join(path_url, shape_name)
#     fn_dir = os.path.join(path_dir, shape_name)
#     if os.path.exists(fn_dir[:-3] + 'shp'):
#         print('%s exsits.')
#         continue
#     else:
#         response = requests.get(fn_url)
#         response.raise_for_status()
#         if len(response.content) == 0:
#             # print('%s is not available (no content).' % shape_name)
#             continue
#         else:
#             f = open(fn_dir,'wb')
#             f.write(response.content)
#             f.close()
#             print(fn_dir, 'is saved.')
#             shutil.unpack_archive(fn_dir, path_dir)
#             os.remove(fn_dir)

In [5]:
country_code1

,COUNTRY,ADMIN0
0,AO,Angola
1,BF,Burkina Faso
2,BI,Burundi
3,CF,Central African Republic
4,SO,Somalia
5,TJ,Tajikistan
6,TZ,Tanzania
7,ZA,South Africa
8,ZW,Zimbabwe
9,ZM,Zambia


### 4. Cross-comparison bewteen FDW data and FEWS NET shapefiles

In [6]:
# Country, ISO code, and Admin levels
code_year = pd.concat([df['country'], df['fnid'].apply(lambda x: x[:2]), df['fnid'].apply(lambda x: x[2:8])],axis=1)
code_year = code_year.drop_duplicates().reset_index(drop=True)
code_year.columns = ['country','code','year']
for i, row in code_year[['country','code']].drop_duplicates().iterrows():
    country, code = row['country'], row['code']
    year = code_year.loc[code_year['code'] == code, 'year']
    print(country, code, sorted(year.values))
code_year['shape'] = False
for i, row in code_year.iterrows():
    country, code, year = row['country'], row['code'], row['year']
    shape_name = '%s_Admin%d_%d.shp' % (code, int(year[-1]), int(year[:4]))
    exist = os.path.exists(os.path.join(path_dir, shape_name))
    if exist == True:
        code_year.loc[i, 'shape'] = exist
    else:
        fn_url = os.path.join(path_url, shape_name[:-3] + 'zip')
        fn_dir = os.path.join(path_dir, shape_name[:-3] + 'zip')
        response = requests.get(fn_url)
        response.raise_for_status()
        if len(response.content) == 0:
            continue
        else:
            f = open(fn_dir,'wb')
            f.write(response.content)
            f.close()
            print(fn_dir, 'is saved.')
            shutil.unpack_archive(fn_dir, path_dir)
            os.remove(fn_dir)
code_year_miss = code_year[code_year['shape'] == False]
reporting_unit = code_year_miss['year'].apply(lambda x: x[-2] == 'R')
code_year_miss = code_year_miss[~reporting_unit].reset_index(drop=True)

Afghanistan AF ['2005A1', '2017A1', '2018A1']
Burkina Faso BF ['1984A1', '2001A2']
Burundi BI ['1991A1', '1998A1']
Central African Republic CF ['2003A1']
Chad TD ['1960A1', '2008A1', '2012A1']
Côte d'Ivoire CI ['2014A0']
Cuba CU ['1976A0', '2010A1']
El Salvador SV ['1841A1']
Ethiopia ET ['1994A1', '1994A2', '2001A1', '2001A2', '2003A1', '2003A2', '2007A2', '2008A1', '2008A2', '2014A1', '2014A2', '2019A2']
Guinea GN ['1990A1', '1990A2']
Haiti HT ['2003A0', '2003A1']
Honduras HN ['1997R1']
Kenya KE ['1982A1', '1982A2', '1989A1', '1989A2', '2013A1']
Lesotho LS ['1980A1']
Liberia LR ['1985A1', '2008A1']
Madagascar MG ['1982A3', '1982R3', '2004A3', '2009A2']
Malawi MW ['1971A2', '1998A2', '2003A2', '2012R3', '2014R3']
Mali ML ['1980A1', '1993A1', '2016A1']
Mauritania MR ['1990A1']
Mozambique MZ ['1990A1']
Nicaragua NI ['1998A1']
Niger NE ['1972A2', '1992A2', '2001A2', '2012A2']
Nigeria NG ['1996A1']
Panama PA ['1980A1', '1983A1', '1997A1', '2014A0', '2014A1']
Paraguay PY ['1992A1']
Rwanda R

In [7]:
code_year_miss

,country,code,year,shape
0,Afghanistan,AF,2018A1,False
1,Côte d'Ivoire,CI,2014A0,False
2,Cuba,CU,1976A0,False
3,Cuba,CU,2010A1,False
4,Haiti,HT,2003A0,False
5,Panama,PA,1980A1,False
6,Panama,PA,1983A1,False
7,Panama,PA,1997A1,False
8,Panama,PA,2014A0,False
9,Panama,PA,2014A1,False
